In [1]:
import pandas as pd
import psycopg2
import json
import datetime
%matplotlib inline

In [206]:
pd.set_option("display.max_colwidth", 500)

In [173]:
db_lexico = psycopg2.connect(**json.load(open('data/db_lexico.json')))

In [174]:
today = datetime.date.today()
weekday = today.weekday()
start_delta = datetime.timedelta(days=weekday, weeks=1)
last_week_start = today - start_delta
last_week_end = last_week_start + datetime.timedelta(days=5)

In [271]:
data = None
for w in reversed(range(4)):
    week_start = last_week_start - datetime.timedelta(days=w*7)
    print(f"week : {week_start}")
    for d in range(5):
        day = week_start  + datetime.timedelta(days=d)
        data_day = pd.read_sql(f'''select * from core_word 
                       where channel_id = 7 
                       and "dateTime" >= '{day.strftime('%Y-%m-%d)')} 18:45:00' 
                       and "dateTime" <= '{day.strftime('%Y-%m-%d)')} 20:20:00' 
                       ''', db_lexico)
        if len(data_day) == 0:
            print(f'KO - {day}')
            continue
        print(day)
        if data is None:
            data = data_day
        else:
            data = data.append(data_day)

week : 2022-02-28
KO - 2022-02-28
KO - 2022-03-01
KO - 2022-03-02
KO - 2022-03-03
KO - 2022-03-04
week : 2022-03-07
KO - 2022-03-07
KO - 2022-03-08
KO - 2022-03-09
KO - 2022-03-10
KO - 2022-03-11
week : 2022-03-14
KO - 2022-03-14
KO - 2022-03-15
KO - 2022-03-16
KO - 2022-03-17
KO - 2022-03-18
week : 2022-03-21
2022-03-21
2022-03-22
2022-03-23
2022-03-24
2022-03-25


In [274]:
len(data)

62574

In [275]:
data = data.sort_values("dateTime")

In [276]:
data["context_before"] = [" ".join(data.word.iloc[max(0, k-50):k].to_list()) for k in range(len(data))]

In [277]:
data["context_after"] = [" ".join(data.word.iloc[k+1:min(k+50, len(data))].to_list()) for k in range(len(data))]

In [278]:
data.sample(5)

,id,dateTime,word,lemma,channel_id,context_before,context_after
9558,171952161,2022-03-21 19:56:43.111603+00:00,en,en,7,de plus en plus beau on peut dire ça oui toi oui comment ça tourne bien moi ça va ma gueule euh marco je trouve donc ce soir parce que ça fait longtemps qu on en parlait tous les deux marco e c est un gars voilà qui a grandi,banlieue il le dit tout le temps et il voulait être en face deux de stanislas l'ego de quelques uns qui voilà qui qui qui soutient éric zemmour alors stanislas rigaud racontez-nous ce qui vous est arrivé c'était vendredi après le meeting d'eric zemmour quoi il attend donc bonjour
4070,176049688,2022-03-23 19:20:34.806988+00:00,jean,jean,7,les syriens euh on va passer aux quarts d'heure sans filtre avant de revenir sur cette rumeur sur emmanuel macron a démenti d'ailleurs euh ce numero norfolk pour ça devient n'importe quoi euh depuis un mois maintenant vous le savez la guerre fait rage en ukraine et le premier ministre,castex annoncer quelque chose cent mille places d'hébergement euh pour euh ben euh pour euh pour les réfugiés ukrainiens alors bien sûr il y a beaucoup de gens qui sont courts et il y a certaines personnes qui ont grincé des dents e junon juillet deux brillance un source
7311,173998425,2022-03-22 19:50:14.699305+00:00,en,en,7,et d'ailleurs c'est grâce à ces personnes-là que chloe gérard garnier remercia gentiment à cette folie tarse remercie aussi également sexuellement c'était ouais ça va ça vous a blessé quand même les choses que vous avez lu ça m'a blessé parce que comme vous étiez je me suis pas reconnue,fait je me suis clairement pas reconnu et on se critiquer quelque inquiétez pas vous critiquer quelqu'un qui était prêt c'était moi mais c'est exactement moi a déjà des personnalités aux alentours restaurant déjà les personnes qui ne va pas faire la différence entre eux la télé et ceux
10745,178218744,2022-03-24 20:02:49.144694+00:00,ça,cela,7,a pas mais quand est-ce qu'elle est là et les charges ont précédé passé qui est grave c'est la forme grave c'est ça qui est grave alors ma revanche si je te passe pas être évident mais que ça t'arrivera rien spécialiste combien de temps maintenant que tu vois c'est,ton problème maintenant c'est que pour toi tout est chaud pour la téléréalité pour toi mais cet horizon qu'en parlant de formes graves mathieu allez vas-y met à jour dans la semaine du vingt-huit vingt-huit février au six mars par exemple sont entrées par jour quatre-vingt-dix-sept personnes en soins
3014,173940899,2022-03-22 19:13:05.590535+00:00,des,de,7,suivant votre programme avec valence qu'est-ce que tu me dis merci on dit quoi merci tu vraiment sympa merci tête un ami alors qu'il est près de moi on discute comme ça on discute un petit peu chiffon e six samedi vingt heures trace on est en direct ce sont,cons qu'ils ferment leurs gueules qui a les chroniqueurs problème soit on s'en prend violemment à jean-marie bigard et fabrice luchini dans une interview choc il était euh on a parti euh voilà euh sur canal plus regarder l'extrait i il est incroyable extrait et jean-marie bigard sera avec


In [279]:
data_words = data.groupby('word').id.count().reset_index().rename({'id': 'count'}, axis=1)

In [280]:
data_words = data_words.sort_values('count', ascending=False)
data_words.head()

,word,count
1794,de,1747
5416,que,1289
3324,je,1211
2409,et,1160
3699,la,1128


In [281]:
data_words['len'] = [len(w) for w in data_words.word]

In [282]:
data_words[data_words['len'] > 5].head(10)

,word,count,len
7231,vraiment,105,8
6783,toujours,94,8
2400,est-ce,86,6
594,beaucoup,86,8
459,aujourd'hui,84,11
3943,maintenant,82,10
2710,france,81,6
5430,question,67,8
4243,moment,64,6
5067,pourquoi,64,8


In [285]:
keywords = ['biodiversité', 'climatique', 'climat', 'giec', 
            'présidentielle', 'élection',
            'réchauffement', 'dérèglement',
            'zemmour', 'jadot', 'macron', 'pécresse', 'mélenchon', 'hidalgo',
            'guerre', 'ukraine', 'sécurité', 'immigration', 'impôts', 'chasse',
            'nucléaire', 'éolienne' 'éoliennes']

In [286]:
data_words[[x.lower() in keywords for x in data_words.word]][['word', 'count']]

,word,count
3920,macron,59
2905,guerre,58
7302,zemmour,34
6953,ukraine,25
4349,mélenchon,24
5282,présidentielle,16
5340,pécresse,16
2989,hidalgo,15
6549,sécurité,14
3103,impôts,4


In [289]:
focus = 'chasse'

In [290]:
data[data.word == focus]

,id,dateTime,word,lemma,channel_id,context_before,context_after
6768,171917222,2022-03-21 19:35:15.413343+00:00,chasse,chasse,7,il veut liberté ma vie meilleur programme est infime inserm deux mille vingt-deux vite en ce moment offre découverte sur w w quand il y a les gens qui rêvent de changer de vie et a ceux qui comme le décide de le faire c'est quand même un avion de,jeanne quand tu penses trop à la française moi je te laisse pas rester ici toute seule ce fait c'est vrai c'est pas un anniversaire trois années d'abstinence ça suffit disponible sur prime video commencez votre essai gratuit de trente jours moi j'ai repris le pouvoir sur mon crédit
9717,171953619,2022-03-21 19:57:21.899430+00:00,chasse,chasse,7,haine en sortant du meeting avec des nécessite un autre porte-parole de la campagne et on rentre tranquillement à la gare et en fait des bandes de voyous racailles euh je crois que ça pas de trop d'autres mots pour qualifier ça sont venus en fait ça une sorte de,aux personnes sortant du meeting d'eric zemmour et demander à ce qui s'est passé est-ce que vous étiez en médecine d'eric zemmour est-ce que tu votes pour eric zemmour etc les premières personnes que je croise me dit ça etc donc se marient ont deux meetings eux et ils
3103,173940988,2022-03-22 19:13:37.480535+00:00,chasse,chasse,7,l'extrait i il est incroyable extrait et jean-marie bigard sera avec nous dans un instant pour lui répondre nous dire si vous pensez vous que des personnes voilà euh des des personnalités doivent donner leur avis euh voilà parce que pour françois cluzet sinon voilà après moi je vois la,et partagée là-dessus vous nous donnerez votre avis raymond e donne plus le faire dans un instant merci retour c'est un nouveau coup de gueule signé françois cluzet mais ferme ta gueule l'acteur qui ne mâche pas ses mots qui a son franc-parler eux bien sûr vous vous en
11488,174036482,2022-03-22 20:17:18.012134+00:00,chasse,chasse,7,août hum c'est très très très très bon mari c'est déjà fait et très bien fait et toutes nos recettes sont sans conservateurs il y a les gens qui aiment changer de vie il y a ceux qui comme le récit de le faire c'est quand même un avion de,jeanne quand tu penses trop à la française fenêtre soit rester si tout ça se fait c'est c'est pas un anniversaire trois années d'abstinence ça suffit disponible sur prime video commencez votre essai gratuit de trente jours je suis désolé monsieur et madame se sont déjà morts voilà boule


In [157]:
focus_time = data[data.word == focus].dateTime.iloc[0]
focus_time

Timestamp('2022-03-23 21:18:37.468051+0000', tz='UTC')

In [158]:
data_context = data[(data.dateTime >= (focus_time-datetime.timedelta(minutes=1))) 
      & (data.dateTime < (focus_time+datetime.timedelta(minutes=1)))
    ].sort_values('dateTime')

In [159]:
context_before = " ".join(data_context[data_context.dateTime < focus_time].word.to_list())

In [160]:
context_after = " ".join(data_context[data_context.dateTime > focus_time].word.to_list())

In [161]:
context_before

"c'est que moi je pense que la france elle se choisit elle se mérite elle s'aiment mais je veux pas l'immigration zéro je veux l'immigration contrôlée je dis pas aux étudiants étrangers vous viendrez plus en france je dis pas eux qui aura plus d'immigration travail boris johnson il a essayé hein et en angleterre il a tenu trois mois il y avait plus personne pour conduire les camions que le sujet aujourd'hui c'est des quotas par métier par payer et je revendique de dire que tous les pays posent pas les mêmes problèmes et que on a aujourd'hui des pays avec lesquels on a des très très bonnes relations il aucun problème migratoire avec et y a des pays avec lesquels on a un problème le mali par exemple le mali nos soldats cinquante-huit soldats morts qui ont payé le prix du sang au mali cinquante-huit cela et eux les maliens refusent de reprendre leur clandestins ben eh ben on dira zéro visas pour le mali voilà parce que on peut pas à la fois payé le prix du sang aider au développement du m

In [162]:
focus

'immigration'

In [163]:
context_after

'zéro comme vous et j ai entendu eric zemmour alors vous vous avancez un peu masqué mme maréchal si je peux me permettre basique mais je ne sais pas je sais pas si nous marchons mais à mme maréchal avance masqué on dirait que le pain un peu marre'

In [272]:
full_text = " ".join(data[data.dateTime < '2022-03-22'].word)

In [273]:
full_text[:5000]

"le bon pour tous réservé maintenant cette semaine c'est le plaisir de manger sainement grâce à la machine solide à vingt-quatre quatre-vingt-dix conserver la saveur et les vitamines de vos aliments en quelques secondes éviter les gaspillages et faites des économies en conservant les aliments plus longtemps la médecine solide ben seulement vingt-quatre euros quatre-vingt-dix c'est ici j féliciter jimmy c'est fou comme notre nature l'emporte toujours vivre ensemble intensément contre vents et marées et si ça finit mal de gorge peu intense et sans fièvre pas paix vous accompagne l'action antibactérienne de lidocaïne sait-il pyrénéen misogyne combat localement l'infection et soulage la gorge lisa haynes combattez le mal de gorge par votre nature tout médicament peut exposer à des risques parlez-en à votre pharmacien casting cinquante en faveur du suspens cinquante enfin peut-être pas euh par contre on a pas de cadeaux haha d on a besoin cette année c'est moi qui fait les cadeaux cadeau cl